In [1]:
### web scraper, information extracter

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import numpy as np
import re, sys, random, glob, json, unidecode, unicodedata
import ipdb, time, sys, os, pickle
from IPython.core.display import display, HTML
from IPython.display import clear_output
from urllib.request import urlopen
from collections import Counter
from bs4 import BeautifulSoup  

display(HTML("<style>.container { width:100% !important; }</style>"))

# Matplotlib Configuration
pylab.rcParams.update({'legend.fontsize': 'medium',
                       'font.size'      : 12,
                       'figure.figsize' : (10, 10),
                       'axes.labelsize' : 'medium',
                       'axes.titlesize' : 'medium',
                       'axes.grid'      : 'on',
                       'xtick.labelsize': 'medium',
                       'ytick.labelsize': 'medium'})

def print_runtime(start: float, printer: bool = True) -> None:
    end = time.time()
    if printer:
        print(f'Runtime: {int((end - start) // 60)} min {int((end - start) % 60):2d} sec')
    else:
        return f'({int((end - start) // 60)} min {int((end - start) % 60):2d} sec)'


with open('urls.txt', 'r') as f:
    urls = f.readlines()
urls = [url[:-1] for url in urls]

with open('scraped_data_1.json', 'r') as f:
    data = json.load(f)
with open('scraped_data_2.json', 'r') as f:
    data.update(json.load(f))
with open('scraped_data_3.json', 'r') as f:
    data.update(json.load(f))

results = []
dump = dict()
idx_start = 330
len(data) # 42% of urls are dud!

/var/folders/jf/qykqz6ks0vs56cb_zyf_7x_00000gn/T/ipykernel_73469/4190775106.py:9: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


1222

In [7]:
# prompt1= """I'll give you an HTML tag. Estimate the likelihood that this tag leads to a bio page of a business professional or a team, """+\
# """from which named entities such as names, contact information, emails, and phone numbers can be extracted. Provide your estimate as a """+\
# """single number between 0 and 100, where 0 means very unlikely, and 100 means absolutely certain. Do not include any other text or explanation """+\
# """in your response. Here's the HTML tag:""" + a_tag

# completion = client.chat.completions.create(
#   model=model,
#   messages=[
#     {"role": "system", "content": "You are an assistant that extracts named entities from provided text, and returns an answert that's only comprised of " +\
#      "json formatted text"},
#     {"role": "user", "content": prompt}
#   ]
# )

# content = completion.choices[0].message.dict()['content']


In [15]:
# prompt2 = """Here's some text I extracted from a webpage:\n""" + text +\
# """ \n Extract the following Named Entities from the above text: "First Name", "Middle Name", "Last Name", "Profession", """ +\
# """Company Name", "Phone Number", "Email", "Address" , "State", "Zipcode", "Bio", "Specialty Areas" """ +\
# """and  put them in a json file with the keys I specified. If the information is missing """ +\
# """leave the value of that key blank. Only return a JSON formatted text. Don't say anything else, don't write any python code. """ +\
# """If any of the values in the Json file is a list of strings, make sure you wrap the items in square brackets compatible with python syntax"""

prompt_2 = """I have a text excerpt extracted from a webpage containing various details about a business professional. Based on this text, please identify and organize """+\
"""the following named entities into a JSON format: First Name, Middle Name, Last Name, Profession, Company Name, Phone Number, Email, Address, """+\
"""State, Zipcode, Bio, and Specialty Areas. If any information is not available from the text provided, leave the corresponding value in the JSON """+\
"""blank. Additionally, if a named entity (like 'Specialty Areas' or others) consists of multiple items, please format it as a list in Python-compatible syntax """+\
"""(using square brackets). Remember to return only the JSON formatted text without any additional comments or Python code. \nHere's the text: """ + text
print(prompt_2)

I have a text excerpt containing various details about a business professional. Based on this text, please identify and organize the following named entities into a JSON format: First Name, Middle Name, Last Name, Profession, Company Name, Phone Number, Email, Address, State, Zipcode, Bio, and Specialty Areas. If any information is not available from the text provided, leave the corresponding value in the JSON blank. Additionally, if a named entity (like 'Specialty Areas' or others) consists of multiple items, please format it as a list in Python-compatible syntax (using square brackets). Remember to return only the JSON formatted text without any additional comments or Python code. 
Here's the text: https://ibgbusiness.com/


In [16]:
from openai import OpenAI
client = OpenAI()


start = time.time()
model = "gpt-3.5-turbo"


for idx, url in enumerate(urls[idx_start:], start=idx_start):
    s0 = time.time()
    
    try:
        if url not in data:
            print(f'  ===> url:{url} not in data {print_runtime(start, False)}')
            continue

        text = data[url]
        if "  I had an exception" in text:
            print(f'  ===> {text} {print_runtime(start, False)}')
            continue

        prompt = """Here's some text I extracted from a webpage:\n""" + text +\
        """ \n Extract the following Named Entities from the above text: "First Name", "Middle Name", "Last Name", "Profession", """ +\
        """Company Name", "Phone Number", "Email", "Address" , "State", "Zipcode", "Bio", "Specialty Areas" """ +\
        """and  put them in a json file with the keys I specified. If the information is missing """ +\
        """leave the value of that key blank. Only return a JSON formatted text. Don't say anything else, don't write any python code. """ +\
        """If any of the values in the JSON file is a list of items, make sure you wrap the items in square brackets compatible with python syntax."""

        completion = client.chat.completions.create(
          model=model,
          messages=[
            {"role": "system", "content": "You are an assistant that extracts named entities from provided text, and returns an answert that's only comprised of " +\
             "json formatted text"},
            {"role": "user", "content": prompt}
          ]
        )

        content = completion.choices[0].message.dict()['content']
        if ",}" in content:
            print(f"  ===> replacing 1  {print_runtime(start, False)}")
            content = content.replace(',}', '}')
        if ",\n" in content:
            print(f"  ===> replacing 2 ',\n' with '\' {print_runtime(start, False)}")
            content = content.replace(",\n", ",")
            
        json_output = json.loads(content)
        json_output['url'] = url
        results.append(json_output)
        print(f'finished round idx: {idx} len(results)={len(results)} {print_runtime(start, False)} \n')

    except Exception as e:
        print(f'---> at idx:{idx} had Exception {e} {print_runtime(start, False)}')
        dump[url] = content
        
        
print_runtime(start)

  ===> replacing 2 ',
' with '' (0 min  6 sec)
finished round idx: 330 len(results)=1 (0 min  6 sec) 

  ===> replacing 2 ',
' with '' (0 min 17 sec)
finished round idx: 331 len(results)=2 (0 min 17 sec) 



KeyboardInterrupt: 

In [231]:
# with open('idx_dump_results.pkl', 'wb') as f:
#     pickle.dump((idx, dump, results), f)
    

In [202]:
import pandas as pd
df = pd.DataFrame(results)
df.to_csv('named_entities.csv')
df


,First Name,Middle Name,Last Name,Profession,Company Name,Phone Number,Email,Address,State,Zipcode,...,,First_Name,Middle_Name,Last_Name,Company_Name,Phone_Number,Specialty_Areas,Ebook_Title,Ebook_Link,Hours
0,John,,Lubinski,Commercial Realtor,Hanna Real Estate,330-975-2280,,,Ohio,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,John,,Geiwitz,Business Broker,"Transworld Business Brokers of North Florida, ...",904-412-5771,johng@tworld.com,"7545 Centurion Parkway, Suite 406",FL,32256,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Cynthia,M.,Daughtrey,Esq.,3D Real Estate,303-548-9659,cynthia@3drealestate.co,2679 Main St,CO,80120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,,,,Business Broker,Advanced Business Brokers,269-903-8143,,2615 Hill an Brook Dr,MI,49024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,,,,Florida Business Broker,Acquisition Experts LLC,(855) 965-1301,broker@acquisitionexperts.net,50 SE Ocean Blvd Suite 205,FL,34994,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,,,,Investments,Best Choice Investments,(386) 316-0052,,P.O. Box 291615,FL,32129,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
178,Chase,,Keller,,Better Commercial,319.231.1160,Chase@bettercommercial.co,8705 Chambery Blvd Suite 100,IA,50131,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
179,,,Conley,,BIB Group,2604833199,bibgroupmail@gmail.com,,IN,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
180,,,,,Bison Business,210-774-6800,INFO@BISONBUSINESS.COM,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
